In [12]:
import os
from grpc import server
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re, time
from os import path
import speech_recognition as sr
import socket, threading

Q_TKN = '<usr>'
A_TKN = '<sys>'
BOS = "</s>"
EOS = "</s>"
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

def collateBatch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

# SKT가 학습시켜놓았던 koGPT2 모델과 토크나이저를 들고 옴
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', bos_token = BOS, eos_token = EOS, unk_token = '<unk>', pad_token = PAD, mask_token = MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# 우리 ChatBotData.csv(게임 대화 데이터셋)을 들고와서
Chatbot_data = pd.read_csv("ChatbotData.csv")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_set = ChatbotDataset(Chatbot_data, max_len = 40)
train_dataloader = DataLoader(train_set, batch_size = 32, num_workers = 0, shuffle = True, collate_fn = collateBatch, )


# 저장된 모델(전이학습까지 끝난)이 있으면 들고오는건데 구현 안 했음
#if path.exists('./model.asdf'):
#    print('load model')
#    ChatbotModel().to(device)
#    
#else:
# 추가적인 학습(전이학습)
#model.to(device) # cuda를 잡는 과정에서 계속 gpu와 cpu를 혼선해서 잡아서 주석처리함
model.train()

# 학습을 위한 파라미터 정의
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

print('training start')
for epoch in range(epoch):
    # Epoch 값 만큼 for 돌리기
    print('epoch round ' + str(epoch))
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print('training over')
'''
def comm(client_sock, id):
    # 소켓으로 클라이언트랑 대화
    client_sock.settimeout(3)
    chk = 0
    while True:
        if chk > 20: break
        q = ''
        try:
            data = client_sock.recv(1024)
            nowdir = os.getcwd()
            with open(nowdir + '\\' + 'clientvoice' + str(id) + '.wav', 'wb') as f:
                # 음성파일을 받아와 저장
                try:
                    while data:
                        f.write(data)
                        data = client_sock.recv(1024)
                        print('-')
                except Exception as e:
                    print(e)
                    if e != 'timeout': chk += 1
                    if chk > 20: break
            # 오디오 파일 분석
            r = sr.Recognizer()
            with sr.AudioFile('clientvoice' + str(id) + '.wav') as source: audio = r.record(source, duration = 120)
            stt = r.recognize_google(audio_data = audio, language = 'ko')
            # 분석받은 문자열을 디코드
            data = stt.encode()
            q = data.decode('utf-8')
            q = q.strip() + '?'
            print(q)
            a = ''
            start = time.time()
            while True:
                if time.time() - start > 10:
                    # if 무한루프(10초 이상 분석 중일때)
                    a = '다시 한 번 말씀해주시겠어요?'
                    break
                # 대답 만들기
                input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + '0' + A_TKN + a)).unsqueeze(dim = 0)
                pred = model(input_ids)
                pred = pred.logits
                gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim = -1).squeeze().numpy().tolist())[-1]
                if gen == EOS: break
                print(gen)
                a += gen.replace('▁', ' ')
            a = a.strip()
            a = a.replace('▁', ' ')
            if a == '': a = '잘 모르겠어요.'
            client_sock.sendall(len(a.encode('utf-8')).to_bytes(4, byteorder='big'))
            client_sock.sendall(a.encode('utf-8'))
            chk = 0
        except Exception as e: 
            print(e)
            chk += 1
            if chk > 20: break
        
# 서버 소켓 생성
server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_sock.bind(('', 4444))
# 소켓 리슨 상태로 들어감
server_sock.listen()
i = 0
while True:
    # 클라한테서 소켓 받으면
    client_sock, addr = server_sock.accept()
    i += 1
    # 클라이언트 한 명 마다 쓰레드 하나씩
    th = threading.Thread(target=comm, args=(client_sock, i,))
    th.start()
'''
'''while True:
    q = input('user >').strip()
    a = ''
    if q == 'exit': break
    while True:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + '0' + A_TKN + a)).unsqueeze(dim = 0)
        pred = model(input_ids)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim = -1).squeeze().numpy().tolist())[-1]
        if gen == EOS: break
        a += gen.replace('▁', ' ')
    print('chatbot >' + a.strip())

        '''

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


training start
epoch round 0


KeyboardInterrupt: 